# Running an ANUGA Simulation: Automated Workflow

This notebook demonstrates a **script-based workflow** for setting up, running, and analysing an **ANUGA** simulation.

The emphasis is on:
- reproducibility
- clarity of model setup
- separation between configuration, execution, and analysis

This notebook is intended as a **teaching and development template**.  
Once the workflow is stable, the code will be cleaned up and reorganised into a standalone Python script.


In [ ]:
# Core Python
import os
import sys
from pathlib import Path

# Scientific stack
import numpy as np
import matplotlib.pyplot as plt

# ANUGA
import anuga

print("ANUGA version:", anuga.__version__)

## Model Configuration

In this section we define:
- working directories
- output locations
- basic model parameters

All paths are defined explicitly to support reproducibility.


In [ ]:
# Project directories
PROJECT_ROOT = Path.cwd().parent
DATASTORE = PROJECT_ROOT / "DATASTORE"
OUTPUTS = DATASTORE / "outputs"

OUTPUTS.mkdir(parents=True, exist_ok=True)

# Simulation parameters (placeholders)
SIMULATION_NAME = "anuga_demo"
FINAL_TIME = 3600.0        # seconds
YIELD_STEP = 60.0          # seconds

print("Project root:", PROJECT_ROOT)

## Domain and Mesh

Here we define the computational domain and generate a mesh.
This example uses a very simple rectangular domain for demonstration.


In [ ]:
# Define a simple rectangular domain
boundary_polygon = [
    (0.0, 0.0),
    (1000.0, 0.0),
    (1000.0, 1000.0),
    (0.0, 1000.0)
]

# Create domain
domain = anuga.Domain(boundary_polygon, mesh_filename="demo_mesh.msh")

domain.set_name(SIMULATION_NAME)
domain.set_default_order(2)
domain.set_time(0.0)

print("Domain created")

## Topography and Physical Parameters

This section defines elevation, friction, and other physical properties.
In real applications, these may be read from files.


In [ ]:
# Set a flat bed as a placeholder
def elevation(x, y):
    return np.zeros_like(x)

domain.set_quantity('elevation', elevation)
domain.set_quantity('friction', 0.03)

print("Elevation and friction set")

## Boundary Conditions

Boundary conditions define how water enters and leaves the domain.
This example uses simple reflective and transmissive boundaries.


In [ ]:
# Boundary tags
domain.set_boundary_tags({
    'left':   [0],
    'right':  [1],
    'top':    [2],
    'bottom': [3]
})

Br = anuga.Reflective_boundary(domain)
Bt = anuga.Transmissive_boundary(domain)

domain.set_boundary({
    'left': Br,
    'right': Bt,
    'top': Br,
    'bottom': Br
})

print("Boundary conditions assigned")

## Initial Conditions

Initial water depth and velocity are specified here.


In [ ]:
domain.set_quantity('stage', expression='elevation')
domain.set_quantity('xmomentum', 0.0)
domain.set_quantity('ymomentum', 0.0)

print("Initial conditions set")

## Run Simulation

The simulation is advanced in time using ANUGA's evolve loop.


In [ ]:
for t in domain.evolve(yieldstep=YIELD_STEP, finaltime=FINAL_TIME):
    print(f"Time = {t:.1f} s")

## Basic Output and Visualisation

Here we demonstrate how to access and visualise model outputs.


In [ ]:
# Example: plot water depth at final time
depth = (
    domain.quantities['stage'].centroid_values
    - domain.quantities['elevation'].centroid_values
)

plt.figure(figsize=(6, 5))
plt.tricontourf(domain.triangles, depth, levels=20)
plt.colorbar(label="Water depth")
plt.title("Final water depth")
plt.show()

## Notes and Next Steps

Planned extensions include:
- reading DEMs and spatially varying friction
- inflow hydrographs
- time-varying boundary conditions
- exporting results to files
- refactoring this notebook into a reusable Python script

This notebook serves as a **starting point**, not a complete model.
